# Q-LoRA

### 한국어 QA 시스템 최적화

##### 0. 환경 설정

In [ ]:
!pip install --upgrade typing_extensions

In [1]:
!pip install peft datasets transformers hf_transfer bitsandbytes

In [ ]:
# !python -m pip install --upgrade pip # 위 명령어 실행 시 오류 발생 시 실행

In [ ]:
# !pip install typing-extensions==4.7.1 --upgrade # 위 명령어 실행 시 오류 발생 시 실행

In [ ]:
# !pip install typing_extensions>=4.10.0

In [ ]:
%env CUDA_LAUNCH_BLOCKING=1
%env CUDA_LAUNCH_BLOCKING=1
%env NCCL_DEBUG=WARN
%env CUDA_DEVICE_MAX_CONNECTIONS=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, PeftModel
import bitsandbytes as bnb
import torch.nn.functional as F

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f" 사용 장치: {device}")

 사용 장치: cuda


##### 1. 모델 로드

In [5]:
model_name = "NCSOFT/Llama-VARCO-8B-Instruct"

In [6]:
# 4-bit 양자화된 모델 로드를 하기 위한 설정
from transformers import BitsAndBytesConfig

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # 모델의 가중치를 4-bit 양자화 사용
    bnb_4bit_quant_type='nf4',              # 양자화 타입, nf4는 4-bit 양자화 타입
    bnb_4bit_use_double_quant=True,         # 이중 양자화 사용 여부
    bnb_4bit_compute_dtype=torch.bfloat16   # 계산 데이터 타입
)

In [7]:
# 토크나이저 및 모델 로드

tokenizer = AutoTokenizer.from_pretrained(model_name)

# base 모델에 양자화를 적용해서 진행
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
# gpt, llama 토크나이저는 pad_token이 없음, decoder-only 모델로 만들어졌기 때문에 오류를 방지 하기 위해서 설정해줌
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

##### 2. 데이터 준비

In [9]:
# 데이터셋 로드
dataset = load_dataset('KorQuAD/squad_kor_v1')

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [11]:
dataset['train'][0]

{'id': '6566495-0-0',
 'title': '파우스트_서곡',
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': {'text': ['교향곡'], 'answer_start': [54]}}

In [12]:
def preprocess_data(examples):
    # 입력 데이터 추출 
    inputs = ["질문: " + q + "\n문맥: " + c for q, c in zip(examples["question"], examples["context"])]
    # 정답 데이터 추출
    answer_texts = [a["text"][0] if len(a["text"]) > 0 else "" for a in examples["answers"]]
    # 입력 데이터 토크나이징
    model_inputs = tokenizer(
        inputs,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )
    # 정답 데이터 토크나이징
    labels = tokenizer(
        answer_texts,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )['input_ids']
    # input_ids 기준으로 labels 의 길이를 맞춤
    max_length = model_inputs['input_ids'].shape[1]
    labels = labels[:, :max_length]

    # 패딩된 부분을 -100으로 설정 (loss 계산 시 무시)
    labels[labels == tokenizer.pad_token_id] = -100
    # 토크나이징 데이터에 정답 데이터 추가
    model_inputs['labels'] = labels
    # 토크나이징 데이터 반환
    return model_inputs

In [13]:
# 데이터셋 전처리 적용
train_dataset = dataset['train'].map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)
val_dataset = dataset['validation'].map(preprocess_data, batched=True, remove_columns=dataset['validation'].column_names)

In [14]:
train_dataset[0]

{'input_ids': [103194,
  52688,
  25,
  82818,
  49706,
  105078,
  16969,
  114450,
  112,
  102953,
  21028,
  56069,
  41381,
  54289,
  18918,
  118151,
  35495,
  118947,
  18359,
  108533,
  35495,
  26799,
  107762,
  16969,
  20565,
  5380,
  52688,
  127661,
  25,
  220,
  10750,
  24,
  100392,
  82818,
  49706,
  105078,
  16969,
  114450,
  112,
  102953,
  21028,
  56069,
  41381,
  54289,
  18359,
  114489,
  118151,
  35495,
  55925,
  109842,
  19954,
  109882,
  13094,
  104519,
  234,
  101103,
  117012,
  101228,
  58232,
  17835,
  61816,
  27796,
  105454,
  21028,
  101999,
  104762,
  108047,
  18359,
  108533,
  101103,
  16969,
  118183,
  18359,
  116253,
  121969,
  13,
  23955,
  45618,
  21121,
  82818,
  49706,
  105078,
  16969,
  220,
  10750,
  23,
  116899,
  120997,
  107712,
  108220,
  231,
  43139,
  105178,
  66965,
  24140,
  66965,
  18359,
  50467,
  98272,
  110,
  34804,
  116492,
  103292,
  122691,
  104834,
  54780,
  102326,
  105115,
  1

In [15]:
print(tokenizer.decode(train_dataset[0]['input_ids']))

질문: 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?
문맥: 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [16]:
print(tokenizer.decode(train_dataset[0]['input_ids'], skip_special_tokens=True))

질문: 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?
문맥: 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.


##### 3. 모델 학습 준비

In [17]:
lora_config = LoraConfig(
    r=8,                                    # 모델 학습 시 사용되는 모듈의 수, 저차원의 rank 수
    lora_alpha=32,                          # lora_alpha 값이 클수록 모델 학습 시 사용되는 모듈의 수가 많아짐
    lora_dropout=0.1,                       # 드롭아웃 비율, 과적합 방지
    bias='none',                            # 편향 사용 여부
    target_modules=['q_proj', 'v_proj'],    # 학습할 모듈 지정, 대상이 될 모듈 지정 ('q_proj': 쿼리 프로젝션, 'v_proj': 값 프로젝션)
    task_type='CAUSAL_LM'                   # 학습 타입 지정 (Causal Language Model: 예측 모델)
)

##### target_modules
|  옵션  | 설명 |
|:---------------------:|:--------------------------------------------------------------|
| `'q_proj'`            | Attention의 Query projection. 주로 효율과 성능의 균형을 위해 기본적으로 선택됨. |
| `'v_proj'`            | Attention의 Value projection. 모델 성능 최적화 시, 특히 대형 모델에 권장됨. |
| `'k_proj'`            | Attention의 Key projection. Value projection과 함께 성능 최적화 및 대형 모델에 적합. |
| `'o_proj'`            | Attention Output projection. 성능 향상을 위해 대형 모델에서 사용 가능. |
| `'gate_proj'`         | 게이트 프로젝션. 게이트 구조와 결합하여 추가적인 성능 개선 가능. |
| `'down_proj'`         | 다운 프로젝션. 차원 축소 등 특정 구조에서 성능 및 효율 개선을 위해 활용. |
| `'up_proj'`           | 업 프로젝션. 차원 확장 구조에서 성능 향상 혹은 모델 구조 의도에 따라 적용. |

---
##### task_type
|  옵션      | 설명 |
|:--------------------:|:--------------------------------------------------------------|
| `'CAUSAL_LM'`        | 입력 기반 다음 토큰 예측 (언어모델링/텍스트 생성) 태스크에 사용. |
| `'SEQ_2_SEQ_LM'`     | 입력 시퀀스 → 출력 시퀀스 생성(예: 번역, 요약 등) 태스크에 사용.      |
| `'SEQ_CLS'`          | 시퀀스 전체 분류(예: 감정 분류, 스팸 분류 등) 태스크에 사용.          |
| `'TOKEN_CLS'`        | 각 토큰별 클래스를 예측(예: 개체명 인식) 태스크에 사용.              |
| `'QUESTION_ANSWERING'` | 질문+문맥이 주어질 때 답 예측(질의응답) 태스크에 사용.           |

In [18]:
# 모델에 lora 적용
model = get_peft_model(base_model, lora_config)
# 입력 데이터에 대한 그래디언트 계산 활성화
model.enable_input_require_grads()
# 그래디언트 체크포인팅 활성화
model.gradient_checkpointing_enable()
# 학습 가능한 파라미터 출력
model.print_trainable_parameters()

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.0424


```python
trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.0424
```

위 결과는 현재 로라(LoRA) 방법으로 학습하는 파라미터(가중치)의 개수와 전체 모델 파라미터 중 학습 가능 파라미터가 차지하는 비율을 보여줍니다.

| 항목                | 값 (예시)           | 설명                                                                              |
|---------------------|---------------------|-----------------------------------------------------------------------------------|
| trainable params    | 3,407,872           | 실제로 학습이 진행되는 파라미터 개수.<br>약 340만 개만 업데이트 및 미세조정 됨.      |
| all params          | 8,033,669,120       | 전체 사전학습 모델의 파라미터 총 개수.<br>대부분은 고정(freeze)되어 있음.           |
| trainable %         | 0.0424%             | 전체 파라미터 중 실제 학습이 적용되는 비율.<br>아주 작은 일부만 미세조정 됨.         |

**설명**
- LoRA와 같은 방식은 전체 대용량 파라미터 중 극히 일부만 효율적으로 학습하여, 연산 자원과 메모리를 크게 절약할 수 있음.
- 우리 코드에서는 학습률(learning rate)을 1%로 주어, 학습 대상 파라미터가 빠르게 수렴하도록 설정함.
- LoRA 등 파라미터 효율 방식은 학습률을 비교적 높게 줄 때 효과적이며, 빠른 실험과 수렴에 유리함.


In [19]:
# DataCollatorForSeq2Seq 초기화 → 데이터 셋 전처리 과정에서 사용되는 클래스
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
training_args = TrainingArguments(
    output_dir='./q_lora_korqa',
    save_strategy='epoch',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8, # batch size가 작을때 누산해서 학습을 효과적으로 진행하는 방법
    learning_rate=2e-4,
    weight_decay=0.01,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
    fp16=True,
    push_to_hub=False,             # 허깅페이스 허락 없이 푸시 하면 오류 발생
    report_to='none'               # report 없이 진행
)

In [21]:
# 훈련 프로세스 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # tokenizer=tokenizer,
    data_collator=data_collator
)

##### 4. 모델 학습(Fine-tuning)

In [ ]:
# 훈련 시작
trainer.train()

##### 5. 학습된 모델 활용

(1) 모델 추론

In [22]:
# 설정 저장
from transformers import AutoConfig

trained_model_path = './q_lora_korqa/checkpoint-1888'
# 사전학습된 모델 객체의 정보를 불러온다. 기반 모델의 정보를 불러와서 fine tuning 된 모델 경로에 저장해준다.
config = AutoConfig.from_pretrained(model_name)
config.save_pretrained(trained_model_path)

In [ ]:
# 베이스 모델 학습 결과
from peft import PeftModel
from transformers import pipeline, BitsAndBytesConfig
import torch

# --- 중요 ---
# 훈련 시 사용했던 4-bit 양자화 설정을 *그대로* 다시 정의해야 합니다.
# (노트북 세션이 재시작되었다면 quant_config 객체가 없을 수 있으므로)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 2) 베이스 모델을 16bit로 로드
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,   # 또는 torch.float16
    device_map='auto',
    low_cpu_mem_usage=True
)

# 3) LoRA 로드
model = PeftModel.from_pretrained(
    base_model,
    trained_model_path,
    torch_dtype=torch.bfloat16
)

# 4) pipeline 정의 시 device 고정
qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
) 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

ValueError: The model has been loaded with `accelerate` and therefore cannot be moved to a specific device. Please discard the `device` argument when creating your pipeline object.

In [ ]:
dataset['train'][0]

In [ ]:
question = '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'
context = '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'

# 수정 input_text
input_text = f"질문: {question}\n문맥: {context}\n답변:" # <-- 답변 시작을 유도합니다.

output = qa_pipeline(
    input_text,
    max_new_tokens=50,
    do_sample=False
)
print(output[0]["generated_text"])

(2) HuggingFace Hub